## 1ère exploration lexicométrique
#### Découverte du contenu du fichier des vidéos
#### Application de l'analyse morpho-syntaxique sur les sous-titres des vidéos (notre corpus)
#### Fréquences d'occurrence au sein du corpus et visualisation
#### Distributions selon les sous-corpus (catégories de chaînes, temps...)

#### Installations librairies et modules tiers

In [1]:
# Installations librairies tiers
!pip install spacy-langdetect
!pip install pyldavis
#!pip install swifter

     |████████████████████████████████| 998 kB 12.1 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993430 sha256=5d3cef9a9f9a39692a43d516db01d07d835f5694b1056f9effb5230ee117301c
  Stored in directory: /root/.cache/pip/wheels/89/79/3b/9885ae7f4308f73c514f96d8574d40d7d8173a27731b674013
Successfully built langdetect
     |████████████████████████████████| 1.7 MB 11.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 11.3 MB 34.0 MB/s 
     |████████████████████████████████| 15.7 MB 37.9 MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=25c6b18f290a0793713e45459621b1e54cd4e49fc46824a9d5fc1155e5ae6ba4
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis
  Attem

In [2]:
# ainsi que le modèle moyen français de Spacy
!pip install https://github.com/explosion/spacy-models/releases/download/fr_core_news_md-3.2.0/fr_core_news_md-3.2.0.tar.gz
!pip install fr_core_news_md

     |████████████████████████████████| 46.9 MB 321 kB/s 
     |████████████████████████████████| 6.0 MB 12.6 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 451 kB 48.2 MB/s 
     |████████████████████████████████| 181 kB 30.9 MB/s 
     |████████████████████████████████| 628 kB 41.4 MB/s 
     |████████████████████████████████| 10.1 MB 38.6 MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-3.2.0-py3-none-any.whl size=46938366 sha256=848ef13c4d8c7a98870082879b011b847d28126fd93edc7b194b48783a034655
  Stored in directory: /root/.cache/pip/wheels/ba/71/98/53c0dd09503b9eefd53afb0b60998dba3b2e41759e0b1b41ce
Successfully built fr-core-news-md
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Success

In [ ]:
# ainsi que le modèle transformer français de Spacy
!pip install https://github.com/explosion/spacy-models/releases/download/fr_dep_news_trf-3.2.0/fr_dep_news_trf-3.2.0.tar.gz
!pip install fr_dep_news_trf

     |████████████████████████████████| 400.7 MB 14 kB/s 
     |████████████████████████████████| 51 kB 137 kB/s 
     |████████████████████████████████| 1.2 MB 20.6 MB/s 
     |████████████████████████████████| 2.9 MB 33.3 MB/s 
     |████████████████████████████████| 1.1 MB 38.2 MB/s 
     |████████████████████████████████| 3.3 MB 24.3 MB/s 
     |████████████████████████████████| 596 kB 32.1 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 895 kB 46.0 MB/s 
  Created wheel for fr-dep-news-trf: filename=fr_dep_news_trf-3.2.0-py3-none-any.whl size=400717256 sha256=d0c61800107467f07147359baa9f3820ddb23e77886704237b07fda2ec38c8b1
  Stored in directory: /root/.cache/pip/wheels/b2/05/28/8550d8c03513f467869655ef1676a6a024194fdcd4c31cae12
Successfully built fr-dep-news-trf
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## ICI IL FAUT REDEMARRER L'ENVIRONNEMENT D'EXECUTION (pour employer le modèle spacy)

### Librairies, variables globales...¶

In [1]:
# 1.a
# librairies générales (on ne se servira pas forcément de toutes)
import os
import csv
import re
import math
#import pickle
import datetime as dt
from ast import literal_eval
from tqdm import tqdm 

In [2]:
# 1.b
# librairies générales data science
import numpy as np
import pandas as pd
#import swifter

In [3]:
# 1.c
# pour affichage dtypes pandas quand plus de 60 colonnes
pd.set_option('display.max_rows', 120)
# pour affichage des bars tqdm avec apply et map
tqdm.pandas()

In [4]:
# 1.d
# la librairie graphique la plus simple d'utilisation
import plotly.express as px
import plotly.graph_objects as go

#### librairies text-mining / NLP

In [5]:
# 1.f spacy
import spacy
nlp = spacy.load("fr_core_news_md")

In [6]:
# 1.g
# on ajoute à la pipeline standard de spacy le détecteur de langue
from spacy.language import Language
from spacy_langdetect import LanguageDetector
def get_lang_detector(nlp, name):
    return LanguageDetector()
Language.factory("language_detector", func=get_lang_detector)

<function __main__.get_lang_detector>

In [7]:
# 1.h
nlp.add_pipe('language_detector', last=True)

In [8]:
# 1.i imports spécifiques à google colab
from google.colab import files, drive

#### Liens vers fichiers

In [9]:
# 2.a passage obligé pour récupérer un fichier sur le drive
# il va falloir choisir le drive (account Google) puis renseigner un mot de passe généré 
drive.mount('/content/drive')

Mounted at /content/drive


#### On charge le fichier des vidéos qui comprend aussi les sous-titres

In [ ]:
# 2.b chemin du nom du fichier des vidéos (et sous-titres)
chemin_nom_dataframe = "/content/drive/My Drive/Datasets NLP/Climat/yt_vid_2018-20_climat.csv"

### Exploration des fichiers des vidéos (et sous-titres)

In [ ]:
# 2.c lecture
df_videos = pd.read_csv(chemin_nom_dataframe, sep=',', encoding='utf-8')

In [ ]:
# 2.d
# nombre de vidéos, liste des colonnes (champs) et leurs types
nb_videos = len(df_videos)
print(nb_videos)
df_videos.dtypes

31585


id_video            object
id_chaine           object
titre               object
description         object
tags                object
date_chargement     object
durée              float64
id_yt_catégorie    float64
pays                object
langue              object
active             float64
ss_titres           object
dtype: object

In [ ]:
# 2.e
df_videos.head()

,id_video,id_chaine,titre,description,tags,date_chargement,durée,id_yt_catégorie,pays,langue,active,ss_titres
0,BkJ9WFylmc0,hRE-mEteKamJvcRIB2Seag,Le surf en Sibérie - Papillon Fitness Club - L...,• S’abonner à L’Effet Papillon ► http://www.yo...,"[""L'Effet Papillon"", 'Daphné Roulier', 'canalp...",2018-08-31 18:00:00,735.0,5.0,fr,fr,1.0,[Musique] [Musique] aux confins de la sibérie ...
1,c_imd5HzTYI,4Ku_yoH9xNxU5Nn5ly7XAQ,Made in Europe : une garantie dans nos assiett...,"En Europe, 90% des consommateurs réclament plu...","['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-05-16 09:30:01,759.0,10.0,fr,fr,1.0,nourrir son enfant un geste banal que marina g...
2,8ULgcaxXZjA,4Ku_yoH9xNxU5Nn5ly7XAQ,"Protectionnisme : nos entreprises, perdantes o...","Reportage en Allemagne, où l'excédent commerci...","['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-05-01 09:30:00,764.0,10.0,fr,fr,1.0,région de stuttgart bienvenue dans la vallée d...
3,TBnIXthPoJg,4Ku_yoH9xNxU5Nn5ly7XAQ,Pollution : peut-on en finir avec le charbon ?...,Objectif de l’Union européenne à l’horizon 203...,"['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-04-24 09:30:01,788.0,10.0,fr,fr,1.0,pologne silésie l'une des régions les plus pol...
4,STh2L8LntFQ,4Ku_yoH9xNxU5Nn5ly7XAQ,Attention élèves méchants ! - Vox Pop - ARTE,"Violences scolaires, trop de laisser-faire ? L...","['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-04-09 12:08:30,179.0,10.0,fr,fr,1.0,[Musique] dans ce lycée de toscane certains él...


In [ ]:
# 2.f éliminer colonne(s) inutile(s) - sera remplacée ultérieurement
l_colonnes_a_eliminer = [
'id_yt_catégorie'                    
]
df_videos.drop(l_colonnes_a_eliminer, axis=1, inplace=True)

#### Récupérer les informations sur les chaînes (noms et catégories)

In [ ]:
# 2.g nom du fichier des chaînes
chemin_nom_dataframe = "/content/drive/My Drive/Datasets NLP/noms_chaines.csv"

In [ ]:
# 2.h
df_chaines = pd.read_csv(chemin_nom_dataframe, sep=',', encoding='utf-8')

In [ ]:
# 2.i
# nombre de chaînes listées, liste des colonnes (champs) et leurs types
nb_chaines = len(df_chaines)
print(nb_chaines)
df_chaines.dtypes

1764


id_chaine                   object
nom_chaine                  object
in_corpus                     bool
catégorie_chaine_2019_01    object
dtype: object

In [ ]:
# 2.j
# visualisation des catégories de chaine
df_chaines['catégorie_chaine_2019_01'].value_counts()

Discours militants, alternatifs et engagés                            261
Complotiste confusioniste pamphlétiste                                234
Gilets jaunes                                                         168
Médiation, vulgarisation, décryptage                                  123
Communication de relation publique                                     97
Chaines ou émissions de médias traditionnels                           83
Information locale                                                     82
Autres                                                                 78
Humour, musique, satire                                                70
Informations sélectionnées, collectionnées, détournées, manipulées     56
Conférence                                                             33
Enquête et investigation                                               33
Information spécialisée                                                25
Complotiste confusioniste pamphlétiste

In [ ]:
# 2.k
# ainsi que de la colonne 'in_corpus'
df_chaines['in_corpus'].value_counts()

True     1647
False     117
Name: in_corpus, dtype: int64

In [ ]:
# 2.l
# renommer la colonne des catégories
renommage_colonnes = {'catégorie_chaine_2019_01': 'catégorie_chaine'}
df_chaines.rename(columns=renommage_colonnes, inplace=True)

In [ ]:
# 2.m
# in_corpus ne sert à rien
l_colonnes_a_eliminer = [
'in_corpus'                      
]
df_chaines.drop(l_colonnes_a_eliminer, axis=1, inplace=True)

In [ ]:
# 2.n
# les identifiants de chaine YouTube commencent tous par UC (voir adresses sur le site)
# ce préfixe UC n'apparait pas dans le fichier des vidéos, l'y rajouter
df_videos['id_chaine'] = df_videos['id_chaine'].progress_map(lambda x: 'UC' + x)

100%|██████████| 31585/31585 [00:00<00:00, 667374.08it/s]


In [ ]:
# 2.o
# fusionner les vidéos et les chaînes.
# s'il n'y a pas de chaîne pour la vidéo, on garde quand même la vidéo
df_videos = pd.merge(df_videos, df_chaines,
                     left_on='id_chaine', right_on='id_chaine', how='left') 
print(len(df_videos))
df_videos.dtypes

31585


id_video             object
id_chaine            object
titre                object
description          object
tags                 object
date_chargement      object
durée               float64
pays                 object
langue               object
active              float64
ss_titres            object
nom_chaine           object
catégorie_chaine     object
dtype: object

In [ ]:
# 2.p examiner la répartition des vidéos de notre échantillon selon les catégories de chaîne
df_videos['catégorie_chaine'].value_counts()

Chaines ou émissions de médias traditionnels                          12263
Information locale                                                     4134
Complotiste confusioniste pamphlétiste                                 3167
Discours militants, alternatifs et engagés                             2669
Communication de relation publique                                     2197
Gilets jaunes                                                          1905
Médiation, vulgarisation, décryptage                                   1727
Informations sélectionnées, collectionnées, détournées, manipulées      638
Information spécialisée                                                 525
Enquête et investigation                                                461
Conférence                                                              362
Autres                                                                  226
Humour, musique, satire                                                 136
Chaines ou é

In [ ]:
# 2.q
df_videos.head()

,id_video,id_chaine,titre,description,tags,date_chargement,durée,pays,langue,active,ss_titres,nom_chaine,catégorie_chaine
0,BkJ9WFylmc0,UChRE-mEteKamJvcRIB2Seag,Le surf en Sibérie - Papillon Fitness Club - L...,• S’abonner à L’Effet Papillon ► http://www.yo...,"[""L'Effet Papillon"", 'Daphné Roulier', 'canalp...",2018-08-31 18:00:00,735.0,fr,fr,1.0,[Musique] [Musique] aux confins de la sibérie ...,Les Éclaireurs,Chaines ou émissions de médias traditionnels
1,c_imd5HzTYI,UC4Ku_yoH9xNxU5Nn5ly7XAQ,Made in Europe : une garantie dans nos assiett...,"En Europe, 90% des consommateurs réclament plu...","['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-05-16 09:30:01,759.0,fr,fr,1.0,nourrir son enfant un geste banal que marina g...,Vox Pop - ARTE,Enquête et investigation
2,8ULgcaxXZjA,UC4Ku_yoH9xNxU5Nn5ly7XAQ,"Protectionnisme : nos entreprises, perdantes o...","Reportage en Allemagne, où l'excédent commerci...","['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-05-01 09:30:00,764.0,fr,fr,1.0,région de stuttgart bienvenue dans la vallée d...,Vox Pop - ARTE,Enquête et investigation
3,TBnIXthPoJg,UC4Ku_yoH9xNxU5Nn5ly7XAQ,Pollution : peut-on en finir avec le charbon ?...,Objectif de l’Union européenne à l’horizon 203...,"['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-04-24 09:30:01,788.0,fr,fr,1.0,pologne silésie l'une des régions les plus pol...,Vox Pop - ARTE,Enquête et investigation
4,STh2L8LntFQ,UC4Ku_yoH9xNxU5Nn5ly7XAQ,Attention élèves méchants ! - Vox Pop - ARTE,"Violences scolaires, trop de laisser-faire ? L...","['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-04-09 12:08:30,179.0,fr,fr,1.0,[Musique] dans ce lycée de toscane certains él...,Vox Pop - ARTE,Enquête et investigation


### Sauvegarde en local du fichier créé, pour récupération ultérieure après upload manuel dans le drive **(A PASSER)**

In [ ]:
# 2.r sauvegarde en local du fichier créé
df_videos.to_csv('yt_vid_cha_2018-20_climat.csv', sep=',', encoding='utf-8', header=True,
			           index=False, quoting=csv.QUOTE_ALL)
files.download('yt_vid_cha_2018-20_climat.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Récupération du fichier travaillé

In [ ]:
#3.a passage obligé pour récupérer un fichier sur le drive
# il va falloir choisir le drive (account Google) puis renseigner un mot de passe généré
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# 3.b vérification (progressive)
!ls "/content/drive/My Drive/Datasets NLP/Climat"

'texte référence changement climatique.txt'
 videos_climat_nlp.csv
 yt_com_2018-19_climat.csv
 yt_vid_2018-20_climat.csv
 yt_vid_cha_2018-20_climat.csv
 yt_vid_cha_man_sst_2018-20_climat.csv


In [11]:
chemin_nom_dataframe = "/content/drive/My Drive/Datasets NLP/Climat/yt_vid_cha_2018-20_climat.csv"

In [12]:
# 3.c
df_videos = pd.read_csv(chemin_nom_dataframe, sep=',', encoding='utf-8')

In [ ]:
# 3.d
# nombre de vidéos, liste des colonnes (champs) et leurs types
nb_videos = len(df_videos)
print(nb_videos)
df_videos.dtypes

31585


id_video             object
id_chaine            object
titre                object
description          object
tags                 object
date_chargement      object
durée               float64
pays                 object
langue               object
active              float64
ss_titres            object
nom_chaine           object
catégorie_chaine     object
dtype: object

In [ ]:
# 3.e
df_videos.head()

,id_video,id_chaine,titre,description,tags,date_chargement,durée,pays,langue,active,ss_titres,nom_chaine,catégorie_chaine
0,BkJ9WFylmc0,UChRE-mEteKamJvcRIB2Seag,Le surf en Sibérie - Papillon Fitness Club - L...,• S’abonner à L’Effet Papillon ► http://www.yo...,"[""L'Effet Papillon"", 'Daphné Roulier', 'canalp...",2018-08-31 18:00:00,735.0,fr,fr,1.0,[Musique] [Musique] aux confins de la sibérie ...,Les Éclaireurs,Chaines ou émissions de médias traditionnels
1,c_imd5HzTYI,UC4Ku_yoH9xNxU5Nn5ly7XAQ,Made in Europe : une garantie dans nos assiett...,"En Europe, 90% des consommateurs réclament plu...","['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-05-16 09:30:01,759.0,fr,fr,1.0,nourrir son enfant un geste banal que marina g...,Vox Pop - ARTE,Enquête et investigation
2,8ULgcaxXZjA,UC4Ku_yoH9xNxU5Nn5ly7XAQ,"Protectionnisme : nos entreprises, perdantes o...","Reportage en Allemagne, où l'excédent commerci...","['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-05-01 09:30:00,764.0,fr,fr,1.0,région de stuttgart bienvenue dans la vallée d...,Vox Pop - ARTE,Enquête et investigation
3,TBnIXthPoJg,UC4Ku_yoH9xNxU5Nn5ly7XAQ,Pollution : peut-on en finir avec le charbon ?...,Objectif de l’Union européenne à l’horizon 203...,"['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-04-24 09:30:01,788.0,fr,fr,1.0,pologne silésie l'une des régions les plus pol...,Vox Pop - ARTE,Enquête et investigation
4,STh2L8LntFQ,UC4Ku_yoH9xNxU5Nn5ly7XAQ,Attention élèves méchants ! - Vox Pop - ARTE,"Violences scolaires, trop de laisser-faire ? L...","['ARTE', 'Vox Pop', 'Europe', 'politique', 'éc...",2019-04-09 12:08:30,179.0,fr,fr,1.0,[Musique] dans ce lycée de toscane certains él...,Vox Pop - ARTE,Enquête et investigation


#### examen rapide des colonnes numériques

In [ ]:
# 3.f
df_videos.describe()

,durée,active
count,31558.000000,31585.000000
mean,832.146714,0.961659
std,1575.472929,0.192021
min,5.000000,0.000000
25%,128.000000,1.000000
50%,310.000000,1.000000
75%,794.750000,1.000000
max,39255.000000,1.000000


#### examen rapide de quelques colonnes catégorielles

In [ ]:
# 3.g
df_videos['nom_chaine'].value_counts()

RTL - Toujours avec vous !            1898
Europe 1                              1413
CNEWS                                 1146
RT France                             1081
LE VRAI MONDE ,LES VRAIES AFFAIRES     976
                                      ... 
frenchpatriot1006                        1
Marie Reno                               1
jayslem officiel                         1
L'écho du peuple                         1
Les Éclaireurs                           1
Name: nom_chaine, Length: 1063, dtype: int64

In [ ]:
# 3.h ou pour tout voir
comptages_par_chaine = df_videos['nom_chaine'].value_counts()
l_comptages_chaines = [(chaine, comptage) for chaine, comptage in comptages_par_chaine.iteritems()]
l_comptages_chaines.sort(key=lambda x: x[1], reverse=True)
for chaine, comptage in l_comptages_chaines:
    print("%s : %d" % (chaine, comptage))

RTL - Toujours avec vous ! : 1898
Europe 1 : 1413
CNEWS : 1146
RT France : 1081
LE VRAI MONDE ,LES VRAIES AFFAIRES : 976
Brest Buzz : 763
FRANCE 24 : 709
euronews (en français) : 618
BFMTV : 542
AFP : 507
RMC : 490
France 3 Nouvelle-Aquitaine : 465
TV5MONDE Info : 457
Télé Matin : 452
Public Sénat : 415
28 minutes - ARTE : 397
France 3 Occitanie : 330
LeHuffPost : 316
Conseil économique social et environnemental : 306
France 3 Bourgogne-Franche-Comté : 303
ARTE Découverte : 291
France 3 Grand Est : 288
Sud Radio : 276
France Inter : 235
TV78 - La chaîne des Yvelines : 234
EDF : 228
Frédérick Moulin : 228
Brut : 223
ademe : 214
Ça Zap Tweet ! : 211
Xerfi Canal : 208
C dans l'air : 188
L'Opinion : 187
Aminégociateur aminegociateur TV : 182
France 3 Provence-Alpes-Côte d'Azur : 180
France 3 Normandie : 179
Le Média : 174
France 3 Auvergne-Rhône-Alpes : 170
Figaro Live : 168
La Quotidienne : 166
Sud Ouest : 164
Le blob, l’extra-média : 162
France 3 Corse ViaStella : 152
Fondation Tara Océa

#### Examen de la chronologie des vidéos

In [ ]:
# 3.i
# déroulement dans le temps
# comme on lit un fichier csv, toute conversion antérieure de string en date(time) est perdue
# doit être refaite ici

# conversion d'une chaine de caractères en un objet datetime
# remarque : les cas mauvais ont été mis à None dans le fichier précédent
# on ne convertit que la date (les 10 premiers caractères)
def convert_str_2_date(str_date):
    date = None
    if pd.isna(str_date) == False:
        date = dt.datetime.strptime(str_date[:10], '%Y-%m-%d')
    return date

# conversion proprement dite
df_videos['date_chargement'] = df_videos['date_chargement'].progress_map(lambda x: convert_str_2_date(x))

100%|██████████| 31585/31585 [00:00<00:00, 67589.18it/s]


In [ ]:
# 3.j
# describe
df_videos[['date_chargement']].describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,date_chargement
count,31585
unique,924
top,2019-03-15 00:00:00
freq,210
first,2018-06-01 00:00:00
last,2020-12-10 00:00:00


In [ ]:
# 3.k
# ou mieux, histogramme de la répartition du nombre de vidéos selon le temps
# données
liste_dates_videos = df_videos['date_chargement'].tolist()
# figure
fig = go.Figure()
fig.add_trace(go.Histogram(x=liste_dates_videos))
fig.show()

#### Examen rapide des éléments textuels

In [ ]:
# 3.l
# examen des titres, tags et descriptions de vidéo
# examen aléatoire sur certaines vidéos
# (essayer quelques numéros entre 0 et nb_videos - 1)
no_video = 10000

print(df_videos.iloc[no_video]['nom_chaine'])
print(df_videos.iloc[no_video]['date_chargement'])
print(df_videos.iloc[no_video]['titre'])
print('==================')
print(df_videos.iloc[no_video]['tags'])
print('==================')
print(df_videos.iloc[no_video]['description'])


LE VRAI MONDE ,LES VRAIES AFFAIRES
2018-10-27 00:00:00
EXCLUSIF, La Californie abrite officiellement certains des "bidonvilles les plus sales du monde"
['nasa', 'clinton', 'trump', 'socialisme', 'communisme', 'elite', 'establishment', 'corruptioin', 'totalitarisme', 'fin du monde', 'rechauffement climatique', 'extra-terrestre', 'fumisterie', 'mensonges', 'pq', 'plq', 'caq', 'cnn', 'foxnews', 'lcn', 'rdi']
fumisterie,mensonge,politiciens criminels exposée,système d'esclavage dans lequel on est prisonnier exposé,la nasa,et plus encore,c'est ça le monde selon jeanez marre.


In [ ]:
# 3.m
# et aussi les sous-titres...
print(df_videos.iloc[no_video]['ss_titres'])

[Musique] alors bonjour à tous et merci d'être là pour la suite de l'informateur 27 octobre 2010 à 17 17 euros hors du québec et oui on est embarqué dans un froid hiver vraiment infernal l'automne est devenu le début de l'hiver imaginez vous voulez voir une période de froid s'installe et des jaunes reçus 40 cm de neige en gaspésie juste pour vous donner une idée maintenant on va le faire un tour chez nos amis des américains surtout nos amis gauchistes socialiste de la californie libéral progressiste qui nous donne qui nous donne les bidonvilles les plus sale du monde on a vraiment dans cette obsession devait le sanctuaire d'état sanctuaire de pays sanctuaire où on nous venons breuil nom porte et nos frontières à tout le monde sans aucun contrôle environ venir les plus misérables du monde on se retrouve la tornade des situations catastrophiques vous l'avez vu on aura on le voit ici maintenant californie libéral ou los angeles et san francisco sont égales avec les pays du tiers-monde on 

In [ ]:
# 3.n
# distribution jointe des durées vidéos et de la longueur des sous-titrages
# constitution de listes pour la représentation graphique (en secondes)
l_durees_videos = df_videos['durée'].tolist()
# pour la longueur, passer par une compréhension de liste et une itération sur la colonne du dataframe
# on prend le nombre de "mots" définis comme chaînes séparées par des blancs
# attention, certains sous-titres très courts et avec nombres sont interprétés comme des float !!
l_longs_ss_titres = [len(str(sous_titre).split()) for ix, sous_titre in df_videos['ss_titres'].iteritems()]
# on pourrait aussi prendre la valeur fournie dans le fichier !
# mettre des points de taille petite pour mieux apprécier concentration des vidéos
fig = go.Figure()
fig.add_trace(go.Scatter(x=l_durees_videos, y=l_longs_ss_titres, mode='markers', marker=dict(size=2)))
fig.update_layout(title='Distribution des durées de vidéos vs nombre de mots sous-titrage', 
                  xaxis_title='durée des vidéos', yaxis_title='nb de mots') 
fig.show()

### Utilisation de spacy

#### principales fonctionnalités

In [13]:
# 4.a
# prendre un texte bien formé
texte =\
"Bon nombre de ces impacts se font déjà sentir au niveau actuel de réchauffement, qui est de plus de 1,2 °C en moyenne au niveau mondial. Le Groupe d'experts intergouvernemental sur l'évolution du climat (GIEC) a publié une série de rapports qui prévoient une augmentation significative de ces impacts quand le réchauffement mondial dépassera 1,5 °C. Un réchauffement supplémentaire augmente également le risque de déclencher des seuils critiques appelés points de basculement."

In [14]:
# 4.b
doc = nlp(texte)

In [17]:
# 4.c
# liste des tokens
for tkn in doc:
    print('"%s", lemme : %s, pos : %s, dep: %s, \t%s' %\
          (tkn.text, tkn.lemma_, tkn.pos_, tkn.dep_,
           #tkn.lemma_, tkn._.lefff_lemma,
           #tkn.pos_, tkn._.melt_tagger, 
           "🛑" if tkn.is_stop == True else ""))

"Bon", lemme : bon, pos : ADJ, dep: case, 	
"nombre", lemme : nombre, pos : NOUN, dep: nsubj, 	
"de", lemme : de, pos : ADP, dep: case, 	🛑
"ces", lemme : ce, pos : DET, dep: det, 	🛑
"impacts", lemme : impact, pos : NOUN, dep: nmod, 	
"se", lemme : se, pos : PRON, dep: expl:comp, 	🛑
"font", lemme : faire, pos : VERB, dep: ROOT, 	🛑
"déjà", lemme : déjà, pos : ADV, dep: advmod, 	🛑
"sentir", lemme : sentir, pos : VERB, dep: xcomp, 	
"au", lemme : au, pos : ADP, dep: case, 	🛑
"niveau", lemme : niveau, pos : NOUN, dep: obl:arg, 	
"actuel", lemme : actuel, pos : ADJ, dep: amod, 	
"de", lemme : de, pos : ADP, dep: case, 	🛑
"réchauffement", lemme : réchauffement, pos : NOUN, dep: nmod, 	
",", lemme : ,, pos : PUNCT, dep: punct, 	
"qui", lemme : qui, pos : PRON, dep: advmod, 	🛑
"est", lemme : être, pos : AUX, dep: cop, 	🛑
"de", lemme : de, pos : ADP, dep: case, 	🛑
"plus", lemme : plus, pos : ADV, dep: advmod, 	🛑
"de", lemme : de, pos : ADP, dep: fixed, 	🛑
"1,2", lemme : 1,2, pos : NUM, dep: dep,

In [18]:
# 4.d
# mais aussi liste des phrases
for phrase in doc.sents:
    print(phrase.text)

Bon nombre de ces impacts se font déjà sentir au niveau actuel de réchauffement, qui est de plus de 1,2 °C en moyenne au niveau mondial.
Le Groupe d'experts intergouvernemental sur l'évolution du climat (GIEC) a publié une série de rapports qui prévoient une augmentation significative de ces impacts quand le réchauffement mondial dépassera 1,5 °C.
Un réchauffement supplémentaire augmente également le risque de déclencher des seuils critiques appelés points de basculement.


In [19]:
# 4.e
# liste des entités nommées
for entite_nommee in doc.ents:
    print("%s (%s)" % (entite_nommee.text, entite_nommee.label_))

Groupe d'experts intergouvernemental (ORG)
GIEC (ORG)


In [20]:
# 4.f
# liste des groupes nominaux
for groupe_nominal in doc.noun_chunks:
    print("%s (tête : %s)" % (groupe_nominal.text, groupe_nominal.root.text))

Bon nombre de ces impacts (tête : nombre)
de réchauffement (tête : réchauffement)
° (tête : °)
C (tête : C)
Le Groupe d'experts intergouvernemental sur l'évolution du climat (GIEC) (tête : Groupe)
une série de rapports qui prévoient une augmentation significative de ces impacts (tête : série)
le réchauffement mondial (tête : réchauffement)
1,5 ° (tête : °)
C (tête : C)
Un réchauffement supplémentaire (tête : réchauffement)
le risque de déclencher des seuils critiques appelés points de basculement (tête : risque)


#### Applications aux sous-titres

In [21]:
# 4.g
# mais avant normaliser les sous-titres
# éliminer [Musique] et chaînes analogues...
# et aussi les espaces en trop
FOND_SONORE = re.compile('\[\w+\]')
ESPACES = re.compile('\s+')
def normalise_sous_titre(texte):
    texte_normalise = re.sub(FOND_SONORE, "", texte)
    texte_normalise = re.sub(ESPACES, " ", texte_normalise)
    texte_normalise = texte_normalise.strip(' \n\r\t\f')
    return texte_normalise

##### créations de listes de résultats de tokenisation

###### Mais avant, comme cela prend beaucoup de temps, se limiter à un échantillon aléatoire

In [22]:
# 4.h
df_echanti = df_videos.sample(1000)

In [26]:
ix = 555
print(df_echanti.iloc[ix]['ss_titres'])

[La musique] ainsi le laboratoire nrmat a été créé en collaboration avec l'université de castro et ce laboratoire est dédié à passer par le changement du monde de la synthèse des bordures à travers la mise en forme et la fabrication d'échantillons jusqu'au dernier électrochimique caractérisation ici dans ce laboratoire nous sommes plus axé sur la recherche sur le matériau du cœur du pile à combustible pour l'électrolyse de votre pile technologie si bien que nous produirons électricité ou hydrogène en fait nous sommes vraiment situés dans ce laboratoire au début de la chaîne signifiant que nous allons commencer par la synthèse des matériaux premier fabricant des cellules et le début de l'électrochimie caractérisations afin que plus tard ces cellules puissent être intégré dans une pile ou dans un système et dans le futur laboratoire de test par exemple ils vont tester les piles à combustible basse température et dans l'offre avant ict afin ils testeront les systèmes à haute température e

In [27]:
# 4.i
# utilisation simple
# indépendamment du reste du dataframe (on ne cherche ici pas à l'enrichir)

# boucler sur le dataframe, et comme c'est long, rajouter un compteur
l_tokens_sous_titres = []
for ix, sous_titre in tqdm(df_echanti['ss_titres'].iteritems(), total=len(df_echanti)):
    l_tokens_sous_titre = []
    doc = nlp(normalise_sous_titre(str(sous_titre)))
    for tkn in doc:
        l_tokens_sous_titre.append(tkn)
    l_tokens_sous_titres.append(l_tokens_sous_titre)

100%|██████████| 1000/1000 [06:23<00:00,  2.61it/s]


In [29]:
print(len(l_tokens_sous_titres))
ix = 10
print(l_tokens_sous_titres[ix][0])

1000
saviez


In [30]:
# 4.j
# là dessus, créer une liste des textes de tokens (termes) avec leur fréquence, triée par fréquence décroissante
# passer par un dictionnaire Python pour effectuer le comptage au fur et à mesure
# ICI ON N'UTILISE PAS ENCORE LES LEMMES, LES STOP-WORDS, ETC. CONTENUS DANS LES tkn, C'EST BRUT DE FONDERIE!
d_frequences_termes = {}
for l_tokens_sous_titre in l_tokens_sous_titres:
    for tkn in l_tokens_sous_titre:
        if tkn.text not in d_frequences_termes:
            d_frequences_termes[tkn.text] = 1
        else:
            d_frequences_termes[tkn.text] += 1
l_frequences_termes = [(terme, frequence) for terme, frequence in d_frequences_termes.items()]
l_frequences_termes.sort(key=lambda x: x[1], reverse=True)

In [31]:
# 4.k
# afficher l'histogramme des fréquence
l_frequences = [frequence for (terme, frequence) in l_frequences_termes]
fig = go.Figure()
fig.add_trace(go.Histogram(x=l_frequences))
fig.show()

In [32]:
# 4.l
# on y verra mieux si on élimine les hapax, voire tous les termes de fréquence "peu fréquents":
# (essayer avec plusieurs valeurs)
frequence_coupure_min = 100
l_frequences = [frequence for (terme, frequence) in l_frequences_termes\
                if frequence > frequence_coupure_min]
fig = go.Figure()
fig.add_trace(go.Histogram(x=l_frequences))
fig.show()


In [33]:
# 4.m
# on y voit mieux si on élimine les hapax, voire les termes de fréquence inférieure à une certaine coupure
# essayer aussi d'éliminer les mots les plus courants
frequence_coupure_min = 100
frequence_coupure_max = 10000
l_frequences = [frequence for (terme, frequence) in l_frequences_termes\
                if frequence > frequence_coupure_min and frequence < frequence_coupure_max]
fig = go.Figure()
fig.add_trace(go.Histogram(x=l_frequences))
fig.show()

In [34]:
# 4.n
# vérification graphique de la loi de Zipf
# on construit une liste des logarithmes des rangs
# et une liste des logarithmes (base 10, pour une meilleure interprétabilité) des fréquences
l_log_rangs_frequences = [(math.log(i+1,10), math.log(frequence,10))\
                          for i, (terme, frequence) in enumerate(l_frequences_termes)]
l_log_rangs = [x[0] for x in l_log_rangs_frequences]
l_log_frequences = [x[1] for x in l_log_rangs_frequences]
fig = go.Figure()
fig.add_trace(go.Scatter(x=l_log_rangs, y=l_log_frequences, mode='markers'))
fig.update_layout(title='Distribution des (logs) de fréquence en fonction des (logs) des rangs', 
                  xaxis_title='(log) rang', yaxis_title='(log) fréquence') 
fig.show()

In [35]:
# 4.o
# afficher les 20 termes les plus fréquents
# la liste a bien été triée en ordre inverse !!
nb_termes_plus_frequents = 20
l_termes_plus_frequents = [terme for i, (terme, frequence) in enumerate(l_frequences_termes)\
                           if i < nb_termes_plus_frequents]
l_plus_hautes_frequences = [frequence for i, (token, frequence) in enumerate(l_frequences_termes)\
                           if i < nb_termes_plus_frequents]
# mais après, pour faire apparaitre les barres les plus longues, il faudra mettre les listes à l'envers
l_termes_plus_frequents.reverse()
l_plus_hautes_frequences.reverse()
fig = go.Figure()
fig.add_trace(go.Bar(x=l_plus_hautes_frequences, y=l_termes_plus_frequents, orientation='h'))
fig.show()

In [36]:
# 4.p d'où le besoin d'éliminer les "stopwords"
# on recommence...
d_frequences_termes = {}
for l_tokens_sous_titre in l_tokens_sous_titres:
    for tkn in l_tokens_sous_titre:
        if tkn.is_stop == True:
            continue
        if tkn.text not in d_frequences_termes:
            d_frequences_termes[tkn.text] = 1
        else:
            d_frequences_termes[tkn.text] += 1
l_frequences_termes = [(terme, frequence) for terme, frequence in d_frequences_termes.items()]
l_frequences_termes.sort(key=lambda x: x[1], reverse=True)


In [37]:
# 4.q
# afficher les 20 termes les plus fréquents
# la liste a bien été triée en ordre inverse !!
nb_termes_plus_frequents = 20
l_termes_plus_frequents = [terme for i, (terme, frequence) in enumerate(l_frequences_termes)\
                           if i < nb_termes_plus_frequents]
l_plus_hautes_frequences = [frequence for i, (terme, frequence) in enumerate(l_frequences_termes)\
                           if i < nb_termes_plus_frequents]
# mais après, pour faire apparaitre les barres les plus longues, il faudra mettre les listes à l'envers
l_termes_plus_frequents.reverse()
l_plus_hautes_frequences.reverse()
fig = go.Figure()
fig.add_trace(go.Bar(x=l_plus_hautes_frequences, y=l_termes_plus_frequents, orientation='h'))
fig.show()

##### Pour la sélection des mots, on utilise aussi les catégories morpho-syntaxiques retenues par spacy (universal dependencies) qui sont :
ADJ	adjective<br>
ADP	adposition<br>
ADV	adverb<br>
AUX	auxiliary<br>
CONJ	conjunction<br>
CCONJ	coordinating conjunction<br>
DET	determiner<br>
INTJ	interjection<br>
NOUN	noun<br>
NUM	numeral<br>
PART	particle<br>
PRON	pronoun<br>
PROPN	proper noun<br>
PUNCT	punctuation<br>
SCONJ	subordinating conjunction<br>
SYM	symbol<br>
VERB	verb<br>
X	other<br>
SPACE	space<br>

In [38]:
# 4.r
# répartition un peu arbitraire entre catégories vides et pleines
# les premières faisant office de stopwords, dans un modèle sacs de mots
CATEGORIES_VIDES = ['ADP', 'AUX', 'CONJ', 'CCONJ', 'DET', 'PART', 'PRON',
                    'PUNCT', 'SCONJ', 'SYM', 'SPACE']
CATEGORIES_PLEINES = ['ADJ', 'ADV', 'INTJ', 'NOUN', 'NUM', 'PROPN', 'VERB', 'X']

In [39]:
# 4.s
# il va falloir éliminer les ponctuations, et tant qu'à faire, les autres catégories vides, 
# et on va travailler avec les lemmes
# on recommence...
d_frequences_termes = {}
for l_tokens_sous_titre in l_tokens_sous_titres:
    for tkn in l_tokens_sous_titre:
        if tkn.is_stop == True:
            continue
        if tkn.pos_ in CATEGORIES_VIDES:
            continue
        if tkn.lemma_ not in d_frequences_termes:
            d_frequences_termes[tkn.lemma_] = 1
        else:
            d_frequences_termes[tkn.lemma_] += 1
l_frequences_termes = [(terme, frequence) for terme, frequence in d_frequences_termes.items()]
l_frequences_termes.sort(key=lambda x: x[1], reverse=True)


In [40]:
# 4.t
# POUR ETRE PROPRE, ON METTRA CELA EN FONCTION (PLUS BAS)
# afficher les 20 termes les plus fréquents
# la liste a bien été triée en ordre inverse !!
nb_termes_plus_frequents = 20
l_termes_plus_frequents = [terme for i, (terme, frequence) in enumerate(l_frequences_termes)\
                           if i < nb_termes_plus_frequents]
l_plus_hautes_frequences = [frequence for i, (token, frequence) in enumerate(l_frequences_termes)\
                           if i < nb_termes_plus_frequents]
# mais après, pour faire apparaitre les barres les plus longues, il faudra mettre les listes à l'envers
l_termes_plus_frequents.reverse()
l_plus_hautes_frequences.reverse()
fig = go.Figure()
fig.add_trace(go.Bar(x=l_plus_hautes_frequences, y=l_termes_plus_frequents, orientation='h'))
fig.show()

In [41]:
# 4.u
# (enfin !) utiliser des fonctions pour ne pas tout répéter
FILTRE_ZERO = 0
FILTRE_STOP = 1
FILTRE_VIDES = 2

def liste_termes_par_frequence(l_tokens_sous_titres,
                               niveau_filtre=FILTRE_ZERO, garde_lemmes=False):

    d_frequences_termes = {}
    for l_tokens_sous_titre in l_tokens_sous_titres:
        for tkn in l_tokens_sous_titre:
            if niveau_filtre >= FILTRE_STOP and tkn.is_stop == True:
                continue
            if niveau_filtre >= FILTRE_VIDES and tkn.pos_ in CATEGORIES_VIDES:
                continue
            forme_terme = tkn.text
            if garde_lemmes == True:
                forme_terme = tkn.lemma_
            if forme_terme not in d_frequences_termes:
                d_frequences_termes[forme_terme] = 1
            else:
                d_frequences_termes[forme_terme] += 1
    l_frequences_termes = [(terme, frequence) for terme, frequence in d_frequences_termes.items()]
    l_frequences_termes.sort(key=lambda x: x[1], reverse=True)

    return l_frequences_termes

In [42]:
# 4.v
def montre_termes_plus_frequents(l_frequences_termes, nb_termes_plus_frequents=20):

    l_termes_plus_frequents = [terme for i, (terme, frequence) in enumerate(l_frequences_termes)\
                               if i < nb_termes_plus_frequents]
    l_plus_hautes_frequences = [frequence for i, (terme, frequence) in enumerate(l_frequences_termes)\
                               if i < nb_termes_plus_frequents]
    # mais après, pour faire apparaitre les barres les plus longues, il faudra mettre les listes à l'envers
    l_termes_plus_frequents.reverse()
    l_plus_hautes_frequences.reverse()
    fig = go.Figure()
    fig.add_trace(go.Bar(x=l_plus_hautes_frequences, y=l_termes_plus_frequents, orientation='h'))
    fig.show()

In [43]:
# 4.w
l_frequences_termes = liste_termes_par_frequence(l_tokens_sous_titres,
                                                 FILTRE_VIDES, True)
montre_termes_plus_frequents(l_frequences_termes, 20)

#### Représentation du nombre de termes du sous-vocabulaire défini par un nombre minimum d'occurrences dans le corpus, ou par la présence dans un nombre minimum de documents

In [ ]:
# 5.a dans ces fonction générales, les textes sont sous forme d'une liste de tokens simples (uniquement le texte)
l_textes_l_tokens = [[tkn.text for tkn in l_tokens] for l_tokens in l_tokens_sous_titres]

In [ ]:
# 5.b
# fonction de calcul pour chaque terme rencontré dans le corpus de sa fréquence
def calcule_frequences_termes(l_textes_l_tokens):
  d_frequences_termes = {}
  for l_tokens in l_textes_l_tokens:
    for token in l_tokens:
      if token not in d_frequences_termes:
        d_frequences_termes[token] = 1
      else:
        d_frequences_termes[token] += 1
  l_frequences_termes = [(terme, frequence) for terme, frequence in d_frequences_termes.items()]
  l_frequences_termes.sort(key=lambda x: x[1], reverse=True)
  return l_frequences_termes

In [ ]:
# 5.c 
# établissement de la liste triée par fréquence des (termes, fréquences)
l_frequences_termes = calcule_frequences_termes(l_textes_l_tokens)
print(len(l_frequences_termes))
print(l_frequences_termes[0], l_frequences_termes[100], l_frequences_termes[1000], l_frequences_termes[10000])

54616
('de', 80278) ('tu', 2306) ('connu', 192) ('entraînent', 10)


In [ ]:
# 5.d
# calculer à partir d'une liste de (terme, frequence) une fonction (un dictionnaire, puis liste)
# qui à chaque fréquence (effectivement présente) associe le nb de termes à cette fréquence 
def calcule_nb_termes_frequences(l_frequences_termes):
  d_nb_termes_frequences = {}
  for (terme, frequence) in l_frequences_termes:
    if frequence not in d_nb_termes_frequences:
      d_nb_termes_frequences[frequence] = 1
    else:
      d_nb_termes_frequences[frequence] += 1
  l_nb_termes_frequences = [(frequence, nb_termes) for frequence, nb_termes in d_nb_termes_frequences.items()]
  l_nb_termes_frequences.sort(key=lambda x: x[0], reverse=True)
  return l_nb_termes_frequences

In [ ]:
# 5.e
l_nb_termes_frequences = calcule_nb_termes_frequences(l_frequences_termes)
print(len(l_nb_termes_frequences))
print(l_nb_termes_frequences[0], l_nb_termes_frequences[10], l_nb_termes_frequences[700], l_nb_termes_frequences[790])

794
(80278, 1) (30816, 1) (94, 14) (4, 2676)


In [ ]:
# 5.f
# on peut aussi en déduire la liste (et aussi dictionnaire) des nb de termes sur fréquence cumulée supérieure ou égale à la limite
# la liste est supposée être triée en ordre inverse des fréquences
def calcul_cumul_nb_termes_frequences(l_nb_termes_frequences):
  l_cumul_nb_termes_frequences = []
  cumul_nb_termes = 0
  for (frequence, nb_termes) in l_nb_termes_frequences:
    cumul_nb_termes += nb_termes
    l_cumul_nb_termes_frequences.append((frequence, cumul_nb_termes))
  # en déduire aussi un dictionnaire, qui sera utilisée par une fonction du recherche de nb de termes en fonction de la fréquence minimum retenue
  d_cumul_nb_termes_frequences = {frequence:cumul_nb_termes for (frequence, cumul_nb_termes) in l_cumul_nb_termes_frequences}
  return l_cumul_nb_termes_frequences, d_cumul_nb_termes_frequences

In [ ]:
# 5.g 
l_cumul_nb_termes_frequences, d_cumul_nb_termes_frequences = calcul_cumul_nb_termes_frequences(l_nb_termes_frequences)
print(len(l_cumul_nb_termes_frequences))
print(l_cumul_nb_termes_frequences[0], l_cumul_nb_termes_frequences[10], l_cumul_nb_termes_frequences[700], l_cumul_nb_termes_frequences[790])

794
(80278, 1) (30816, 11) (94, 1946) (4, 19596)


In [ ]:
# 5.h
# recherche du nb de termes supérieurs à une limite de fréquence
def calcule_nb_termes_avec_frequence_min(d_cumul_nb_termes_frequences, frequence_min):
  freq_reference = frequence_min
  if freq_reference in d_cumul_nb_termes_frequences: 
    return d_cumul_nb_termes_frequences[freq_reference]
  while freq_reference not in d_cumul_nb_termes_frequences:
    freq_reference -= 1
    if freq_reference in d_cumul_nb_termes_frequences: 
      return d_cumul_nb_termes_frequences[freq_reference]
    if freq_reference == 0: return -1

In [ ]:
# 5.i
print(calcule_nb_termes_avec_frequence_min(d_cumul_nb_termes_frequences, 2))

31478


In [ ]:
# 5.j
# représentation graphique de la liste de nombre de termes de fréquence supérieure ou égale à un seuil donné
# construit une liste des logarithmes des fréquences en abscisse, pour représentation plus facile
def montre_nb_termes_frequence_sup_seuil(l_cumul_nb_termes_frequences):
  l_log_frequences = [math.log(frequence,10) for (frequence, cumul_nb_termes)\
                      in l_cumul_nb_termes_frequences]
  l_cumul_nb_termes = [cumul_nb_termes for (frequence, cumul_nb_termes)\
                       in l_cumul_nb_termes_frequences]                        
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=l_log_frequences, y=l_cumul_nb_termes, mode='markers'))
  fig.update_layout(title="Nb de termes dont la fréquence est supérieure au (log) d'une fréquence seuil", 
                    xaxis_title='(log) fréquence seuil', yaxis_title='nb de termes') 
  fig.show()

In [ ]:
# 5.k
montre_nb_termes_frequence_sup_seuil(l_cumul_nb_termes_frequences)

#### Pour la suite des cours, sauvegarder les éléments les plus intéressants des résultats de l'analyse spacy

###### On le fait sur un fichier particulier, très court (762 vidéos), mais dont les textes de sous-titres ont été tous édités manuellement, et non via du speech to text ; et les traitements en cours en sont plus rapide...

In [ ]:
# 6.a 
# nom du fichier des vidéos (plus chaines) avec sous-titres manuels
chemin_nom_dataframe = "/content/drive/My Drive/Datasets NLP/Climat/yt_vid_cha_man_sst_2018-20_climat.csv"

In [ ]:
# 6.b lecture
df_videos = pd.read_csv(chemin_nom_dataframe, sep=',', encoding='utf-8')

###### On crée deux colonnes, une pour les tokens de sous-titres, l'autre pour les entités nommées dans les sous-titres; deux listes par vidéos
###### Tokens : text, i (n° du token dans le document), idx (position du token dans le document, lemma_, pos_ (part of speech), head.i (n° du token qui est tête (gouvernant) dans le token, is_stop (word), ent_type (si entité nommée, son type)
###### Entités nommées : text, start_char, end_char, label_

In [ ]:
# 6.c
# fonction qui pour chaque texte, renvoie les éléments voulus 
# liste des tokens avec leurs attributs les plus intéressants,
# liste des entités nommées,
# et aussi langue du texte
def elements_analyse_nlp(texte, nlp):
  # analyse spacy
  doc = nlp(texte)

  # langue du texte
  langue = doc._.language['language']
  # éléments liés aux tokens
  # on conserve aussi des éléments d'analyse syntaxique de dépendances (tête et relation de dépendance)
  l_elements_tkn = []
  for tkn in doc:
    elements_tkn = (tkn.text, tkn.i, tkn.idx, 
                    tkn.lemma_, tkn.pos_, tkn.head.i, tkn.dep_, tkn.is_stop, tkn.ent_type)
    l_elements_tkn.append(elements_tkn)
  # éléments liés aux entités nommées
  l_elements_ent = []
  for ent in doc.ents:
    elements_ent = (ent.text, ent.start_char, ent.end_char, ent.label_)
    l_elements_ent.append(elements_ent)
    
  return langue, l_elements_tkn, l_elements_ent

In [ ]:
# 6.d
# fonction pour inclure particularités des sous-titres
# pas besoin de normaliser le texte sur ces vidéos précises
def elements_analyse_nlp_sous_titres(sous_titres, nlp):
  return elements_analyse_nlp(sous_titres, nlp) 

In [ ]:
# 6.e
# application de l'analyse nlp sur chaque sous-titres de vidéos 
# et conservation en colonnes des éléments intéressants
df_videos[['langue_sous_titres', 'tkn_sous_titres', 'ent_sous_titres']] = df_videos.progress_apply(\
          lambda x: pd.Series(elements_analyse_nlp_sous_titres(x['ss_titres'], nlp)), axis=1)

100%|██████████| 762/762 [03:21<00:00,  3.78it/s]


In [ ]:
# 6.f
ix = 10
print(df_videos.iloc[ix]['ss_titres'])
print(len(df_videos.iloc[ix]['tkn_sous_titres']))
print(df_videos.iloc[ix]['tkn_sous_titres'])

"Mon avenir est ici, rien ne sert d'aller à l'étranger", sourit Fethi Gueldasmi, agriculteur à Sidi Fredj en Algérie Le figuier de Barbarie a ranimé cette commune autrefois pauvre où beaucoup rêvaient d'émigration Originaire du Mexique, le cactus Opuntia ficus-indica s'est répandu sur le pourtour méditerranéen après la découverte des Amériques au XVe siècle Avec ses grands rameaux aplatis appelés "raquettes", il fait partie du paysage aride des campagnes algériennes, à l'état sauvage ou planté comme haie naturelle La cosmétique et de nouveaux débouchés alimentaires ont dopé cette production ces dernières années, alors qu'il était jusque-là cultivé surtout pour son fruit à la chair goûteuse et sucrée – rapportant à peine de quoi vivre, avec des figues vendues 10 dinars pièce (7 centimes d'euro) L'industrie cosmétique et les magazines de mode européens et américains vantent les propriétés antirides de l'huile de figue de Barbarie, tirée des pépins, et des études scientifiques ont aussi r

In [ ]:
# 6.g
# il faut éliminer les vidéos qui n'ont pas assez de texte, une limite de 20 tokens peut faire sens
NB_TOKENS_MIN = 20
df_videos = df_videos.loc[(df_videos['tkn_sous_titres'].str.len() >= NB_TOKENS_MIN)]
print(len(df_videos))

762


In [ ]:
# 6.h
# et autant éliminer les éventuelles vidéos qui ne sont pas en français
df_videos = df_videos.loc[(df_videos['langue_sous_titres'] == 'fr')]
print(len(df_videos))

762


In [ ]:
# 6.i
# sauvegarde du résultat (sera ensuite copié manuellement dans le drive)
df_videos.to_csv('videos_climat_nlp.csv', sep=',', encoding='utf-8', header=True,
			           index=False, quoting=csv.QUOTE_ALL)
files.download('videos_climat_nlp.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 6.j
# vérification de ce que le chargement 
# et l'étape de retransformation des chaînes analyse de résultats sauvegardés en liste de tuples fonctionne bien
chemin_nom_dataframe = "/content/drive/My Drive/Datasets NLP/Climat/videos_climat_nlp.csv"
df_videos_nlp = pd.read_csv(chemin_nom_dataframe, sep=',', encoding='utf-8')

In [ ]:
# 6.k
# et donc conversion
df_videos_nlp['tkn_sous_titres'] = df_videos_nlp['tkn_sous_titres'].progress_map(lambda x: literal_eval(str(x)))
df_videos_nlp['ent_sous_titres'] = df_videos_nlp['ent_sous_titres'].progress_map(lambda x: literal_eval(str(x)))

100%|██████████| 762/762 [00:00<00:00, 2065.85it/s]


In [ ]:
# 6.l
ix = 10
print(df_videos.iloc[ix]['ss_titres'])
print(len(df_videos.iloc[ix]['tkn_sous_titres']))
print(df_videos.iloc[ix]['tkn_sous_titres'])

"Mon avenir est ici, rien ne sert d'aller à l'étranger", sourit Fethi Gueldasmi, agriculteur à Sidi Fredj en Algérie Le figuier de Barbarie a ranimé cette commune autrefois pauvre où beaucoup rêvaient d'émigration Originaire du Mexique, le cactus Opuntia ficus-indica s'est répandu sur le pourtour méditerranéen après la découverte des Amériques au XVe siècle Avec ses grands rameaux aplatis appelés "raquettes", il fait partie du paysage aride des campagnes algériennes, à l'état sauvage ou planté comme haie naturelle La cosmétique et de nouveaux débouchés alimentaires ont dopé cette production ces dernières années, alors qu'il était jusque-là cultivé surtout pour son fruit à la chair goûteuse et sucrée – rapportant à peine de quoi vivre, avec des figues vendues 10 dinars pièce (7 centimes d'euro) L'industrie cosmétique et les magazines de mode européens et américains vantent les propriétés antirides de l'huile de figue de Barbarie, tirée des pépins, et des études scientifiques ont aussi r